In [1]:
import numpy as np
import json
import glob
import cv2
import pickle
from sklearn import metrics

In [2]:
# Helper function
def generate_solvepnp_parameters(size, landmarks):
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )

    # Grab the 2D coordinates of our six sample points
    image_points = np.array([
        (landmarks[33]['x'], landmarks[33]['y']) ,     # Nose tip
        (landmarks[8]['x'], landmarks[8]['y']),     # Chin
        (landmarks[36]['x'], landmarks[36]['y']),     # Left eye left corner
        (landmarks[45]['x'], landmarks[45]['y']),     # Right eye right corner
        (landmarks[48]['x'], landmarks[48]['y']),     # Left Mouth corner
        (landmarks[54]['x'], landmarks[54]['y'])      # Right mouth corner
    ], dtype="double")
    
    return image_points, camera_matrix

# 3D model points.
model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner
                         
                        ])

dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

In [31]:
# Helper function
def visualize_image(im, rotation_vector, translation_vector, image_points, camera_matrix, label, key):
    # Project a 3D point (0, 0, 1000.0) onto the image plane.
    # We use this to draw a line sticking out of the nose
    (nose_end_point2D, jacobian) = cv2.projectPoints(
        np.array([(0.0, 0.0, 500.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
        )
    iris_right = np.reshape(np.asarray(data[key]['iris_right']), (2, 1))
    iris_left = np.reshape(np.asarray(data[key]['iris_left']), (2, 1))
    for p in image_points:
        cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
    cv2.circle(im, (int(iris_left[0]), int(iris_left[1])), 3, (0, 0, 255), -1)
    cv2.circle(im, (int(iris_right[0]), int(iris_right[1])), 3, (0, 0, 255), -1)
    p1 = ( int(image_points[0][0]), int(image_points[0][1]) )
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )

    # Draw a line connecting the two points. This line must show
    # the direction out of the nose
    cv2.line(im, p1, p2, (500,0,0), 2)
    # Display image
    cv2.imshow(label, im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
file = open('low_score_rf.txt', 'r')
uuids = file.readlines()
print(len(uuids))

12


In [25]:
# Load the dataset
with open('data_cleaned.json') as json_file:
    data = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data)
# Convert python list to np array
keys_all = np.asarray(keys_all)
DATASET_SIZE = 2728

In [89]:
# for uuid in uuids[0:1]:
# # Remove the newline character
# uuid = uuid[:-1]
# print('UUID: ' + uuid)
uuid = 'b676104c35e54ddf9579d67765f6d4a2'

with open('classifiers/rf/' + uuid + '.pickle', 'rb') as f:
    rf_classifier = pickle.load(f)
# with open('classifiers/svm/' + uuid +'.pickle', 'rb') as f:
#     svm_classifier = pickle.load(f)

# Array of keys for this uuid
keys = []
for i in range(DATASET_SIZE):
    key = keys_all[i]
    if(key.split('/')[0] == uuid):
        keys.append(key)
print('Number of examples for this uuid: {}'.format(len(keys)))

X = np.zeros((len(keys), 11, 1))
y = np.zeros(len(keys))
t_vec = np.zeros((len(keys), 3))

for i in range(len(keys)):
    key = keys[i]
    im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
    size = im.shape

    landmarks = data[key]['landmarks']

    image_points, camera_matrix = generate_solvepnp_parameters(size, landmarks)

    # Solve the PnP problem with the parameters specified above
    # and obtain rotation and translation vectors
    (success, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_DLS
        )

    # Data from Architecture #2. Reshape is done for compatibility reasons
    iris_right = np.reshape(np.asarray(data[key]['iris_right']), (2, 1))
    iris_left = np.reshape(np.asarray(data[key]['iris_left']), (2, 1))

    # Data from Architecture #3
    left_vector = np.asarray( (abs(iris_left[0] - landmarks[39]['x']), abs(iris_left[1] - landmarks[39]['y'])) )
    right_vector = np.asarray( (abs(iris_right[0] - landmarks[42]['x']), abs(iris_right[1] - landmarks[42]['y'])) )

    X[i, :] = np.concatenate((rotation_vector, iris_left, iris_right, 
                             left_vector, right_vector), axis=0)
    
    t_vec[i, :] = translation_vector.squeeze()

    # Check if it is positive or negative example
    output = key.split('/')[1]
    if(output == 'positive'):
        y[i] = 1
    elif(output == 'negative'):
        y[i] = 0

X = X.squeeze()

m = X.mean(axis=0)
std = X.std(axis=0)
X_scaled = (X-m)/std

# Metrics for all the examples
y_pred = rf_classifier.predict(X_scaled)
print('Accuracy: {}'.format(metrics.accuracy_score(y, y_pred)))
print(metrics.confusion_matrix(y, y_pred))

Number of examples for this uuid: 60
Accuracy: 0.43333333333333335
[[16  3]
 [31 10]]


In [90]:
failed_indices = (y != y_pred)

In [93]:
for i in range(len(failed_indices)):
    if not failed_indices[i]:
        key = keys[i]
        label = key.split('/')[1]
        im = cv2.imread('dataset/' + key)
        size = im.shape
        
        landmarks = data[key]['landmarks']
        image_points, camera_matrix = generate_solvepnp_parameters(size, landmarks)
        
        visualize_image(im, X[i, 0:3], t_vec[i], image_points, camera_matrix, label, key)

In [83]:
print('False negatives:')
for i in range(len(failed_indices)):
    if y[i] == 1 and y_pred[i]==0:
        print(X[i, 10])
print('True negatives:')
for i in range(len(failed_indices)):
    if y[i] == 1 and y_pred[i]==1:
        print(X[i, 10])

False negatives:
2.8192101992093512
3.25477203956018
2.6060781478881836
2.746704725118775
0.3236083617577208
2.879357924828156
2.0970065410320444
2.22178151057318
1.7875318894019472
2.3511106050931403
2.700253706711976
2.539508196023803
2.940511263333832
0.9772604062006849
1.9619749509371331
2.6457295784583437
3.406033515930176
0.8055892724257205
4.744444443629334
1.307962564321656
4.023964184981139
2.4465326896080626
2.6731181511512148
2.45136345349826
0.5634184617262576
2.108732223510742
1.6958942413330078
2.09460126436673
0.5055019305302721
4.2833295968862615
0.24755804355328337
True negatives:
1.2932024002075195
1.4774847764235233
0.8276224869948123
0.8783782078669446
0.6298829592191169
2.037371378678529
1.7213633610651868
1.569537492898803
1.846873466785155
0.880313946650574
